# 2.1 Transit light curve analysis of WASP-12b 

### Universidad de La Laguna Exo & Exo 2024-2025 exercise 2

Author: [Hannu Parviainen](mailto:hannu@iac.es)<br>
Last updated: 21.4.2025

## Initialisation

In [1]:
%matplotlib inline 

In [2]:
import math as mt
import pandas as pd
import warnings
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.pyplot import subplots, setp, rc, Rectangle
from numpy import argmax, array, median, seterr, floor, percentile
from numpy.random import seed, permutation
from astropy.table import Table
from corner import corner
from astropy import units as u

from pytransit import TransitLPF

seterr('ignore')
seed(0)

<frozen importlib._bootstrap>:488: FutureWarning: pybind11-bound class 'celerite.solver.CholeskySolver' is using an old-style placement-new '__setstate__' which has been deprecated. See the upgrade guide in pybind11's docs. This message is only visible when compiled in debug mode.


### Read in the data

First we need to read in the light curve stored in ``wasp-12b_181227_chromatic_k.fits``. The file has light curves for a single transit observed simultaneously in four passbands (g, r, i, and z_s), but we keep things simple and use only the r-band data. 

In our case, the photometry is saved as binary table extensions in the fits file, one extension per passband. To get the r-band data, we need to read the third HDU of the fits file (the first is the primary HDU, the second the first extension, etc.).

We start the same way as with the RV data and take a look at our data and try plotting it.

In [7]:
tb = Table.read('wasp-12b_181227_chromatic_k.fits', 2)
tb[:5]


time_bjd,flux,flux_rel,flux_trg,flux_ref,baseline,model
float64,float64,float64,float64,float64,float64,float64
2482047.483879058,5.022033159660947e+35,3.6169652236677743e-304,-1.9248156416463776e+230,7.8606667779668375e+28,0.992155762607119,-2.997566927708736e-13
-0.11393437602941865,3.334328209997126e-52,-1.924997476716691e+230,-1.1346130948193718e-13,1.384845663054e-312,-1.9248417581274677e+230,-1.924834784620208e+230
-1.9248721797613576e+230,6.821599153709058e+97,-3.9907724185157175e-13,-1.9248893816210045e+230,-0.1143557606846448,-8.859231378570225e-14,-9.297682383502339e-15
-1.0635588232265188e-13,-1.9249974767055218e+230,-1.9247756194981781e+230,1.9204023695198873e-307,-1.7007920049969775e-13,-1.9248347846202591e+230,-0.11466883116855878
2.9298545271601682e+107,-7.37160220859599e-13,-1.924834784620233e+230,-1.924791470211824e+230,-1.9248244782917978e+230,-0.11413955310103055,4.19186868864403e-32


Now, we can assume the time data is stored in the ``time_bjd`` column and the flux the ``flux`` column. In general, if you'd see a file like this, you should check the file's documentation (if such exists) what column is what, or ask the person who created the file.

In [4]:
tref = floor(tb['time_bjd'].mean())
fig, ax = subplots(figsize=(13,5), sharey=True)
ax.plot(tb['time_bjd'] - tref, tb['flux'], drawstyle='steps-mid')
setp(ax, xlabel=f"Time - {tref:.0f} [BJD]", ylabel='Normalised flux', xlim=tb['time_bjd'][[0,-1]]-tref)
fig.tight_layout()

ValueError: Axis limits cannot be NaN or Inf

The depth of the transit signal gives us information about the planet radius
The duration of the transit signal gives us information about the planet orbit
The shape of the transit signal is related to the stellar disc darkening

## Parameter estimation

First, we create an instance of the log posterior function with the redmost light curve data.

Next, we run the *DE* optimiser for ``de_iter`` iterations to clump the parameter vector population close to the global posterior maximum, use the *DE* population to initialise the *emcee* sampler, and run the sampler for ``mc_iter`` iterations to obtain a posterior sample.

### Initialise the LPF and set the priors

In [26]:
lpf = TransitLPF('WASP-12b', 'r', tb['time_bjd'], tb['flux'])
lpf.ps

[  0 |G| tc             N(μ = 0.0, σ = 0.1)                      [    -inf ..      inf],
   1 |G| p              N(μ = 1.0, σ = 1e-05)                    [    0.00 ..      inf],
   2 |G| rho            U(a = 0.1, b = 25.0)                     [    0.00 ..      inf],
   3 |G| b              U(a = 0.0, b = 1.0)                      [    0.00 ..     1.00],
   4 |P| k2             U(a = 0.0025, b = 0.04)                  [    0.00 ..      inf],
   5 |P| q1_r           U(a = 0, b = 1)                          [    0.00 ..     1.00],
   6 |P| q2_r           U(a = 0, b = 1)                          [    0.00 ..     1.00],
   7 |L| wn_loge_0      U(a = -4, b = 0)                         [    -inf ..      inf]]

Print the model parametrization

Set prior parameters that we know: transit center and orbital period

In [27]:
lpf.set_prior('tc', 'NP', 2458480.65, 0.02)
lpf.set_prior('p', 'NP', 1.0914201, 1.1e-09)

tc transit center 
p orbital period 
rho planet density
b impact parameter (if 0 cross in center of star, if 1 cross in the edge of star)
k radius ratio
q1, q2 limb darkening coefficients
wn average wide noise

In [28]:
lpf.ps

[  0 |G| tc             N(μ = 2458480.65, σ = 0.02)              [    -inf ..      inf],
   1 |G| p              N(μ = 1.0914201, σ = 1.1e-09)            [    0.00 ..      inf],
   2 |G| rho            U(a = 0.1, b = 25.0)                     [    0.00 ..      inf],
   3 |G| b              U(a = 0.0, b = 1.0)                      [    0.00 ..     1.00],
   4 |P| k2             U(a = 0.0025, b = 0.04)                  [    0.00 ..      inf],
   5 |P| q1_r           U(a = 0, b = 1)                          [    0.00 ..     1.00],
   6 |P| q2_r           U(a = 0, b = 1)                          [    0.00 ..     1.00],
   7 |L| wn_loge_0      U(a = -4, b = 0)                         [    -inf ..      inf]]

In [ ]:
lpf.optimize_global(niter=500, npop=50)

Global optimisation:   0%|          | 0/500 [00:00<?, ?it/s]

Prior model:

In [ ]:
lpf.plot_light_curve();

In [33]:
lpf.sample_mcmc(5000, thin=20, repeats=2, label='MCMC sampling')

MCMC sampling:   0%|          | 0/2 [00:00<?, ?it/s]

Run 1/2:   0%|          | 0/5000 [00:00<?, ?it/s]

Run 2/2:   0%|          | 0/5000 [00:00<?, ?it/s]

Posterior model with uncertenties:

In [ ]:
lpf.plot_light_curve('mc')

/opt/anaconda3/envs/exoexo/lib/python3.13/site-packages/pytransit/lpf/transitlpf.py:70: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  setp(ax, xlim=self.timea[[0, -1]] - tref, xlabel=f'Time - {tref:.0f} [BJD]', ylabel='Normalised flux')


(<Figure size 1300x400 with 1 Axes>,
 <Axes: xlabel='Time - -192499747671669098494403703339518863392166925794313665730676798309695534603253704343706177946458351723537486617139418110905291686498435753886430640107895567953742843779904597477054150059100923393516665667488496921291533205251293184 [BJD]', ylabel='Normalised flux'>)

### Analysis: overview

The MCMC chains are now stored in ``lpf.sampler.chain``. Let's first have a look into how the chain populations evolved to see if we have any problems with our setup, whether we have converged to sample the true posterior distribution, and, if so, what was the burn-in time.

In [ ]:
fig, axs = subplots(2,4, figsize=(13,5), sharex=True)
ls, lc = ['-','--','--'], ['k', '0.5', '0.5']
percs = [percentile(lpf.sampler.chain[:,:,i], [50,16,84], 0) for i in range(8)]
[axs.flat[i].plot(lpf.sampler.chain[:,:,i].T, 'k', alpha=0.01) for i in range(8)]
[[axs.flat[i].plot(percs[i][j], c=lc[j], ls=ls[j]) for j in range(3)] for i in range(8)]
setp(axs, yticks=[], xlim=[0,5000//20])
fig.tight_layout()

Ok, everything looks good. The 16th, 50th and 84th percentiles of the parameter vector population are stable and don't show any significant long-term trends. Now we can flatten the individual chains into one long chain ``fc`` and calculate the median parameter vector.

In [42]:
fc = lpf.sampler.chain.reshape([-1,lpf.sampler.chain.shape[-1]])
mp = median(fc, 0)

Let's also plot the model and the data to see if this all makes sense. To do this, we calculate the conditional distribution of flux using the posterior samples (here, we're using a random subset of samples, although this isn't really necessary), and plot the distribution median and it's median-centred 68%, 95%, and 99.7% central posterior intervals (corresponding approximately to 1, 2, and 3$\sigma$ intervals if the distribution is normal). 

In [43]:
flux_pr = lpf.flux_model(fc[permutation(fc.shape[0])[:1000]])
flux_pc = array(percentile(flux_pr, [50, 0.15,99.85, 2.5,97.5, 16,84], 0))

In [44]:
zx1,zx2,zy1,zy2 = 0.69,0.71, 0.983, 0.992
tref = floor(lpf.timea.min())
fig, ax = subplots(1,1, figsize=(13,4))
ax.errorbar(lpf.timea-tref, lpf.ofluxa, 10**mp[7], fmt='.', c='C1', alpha=0.75)
[ax.fill_between(lpf.timea-tref,*flux_pc[i:i+2,:],alpha=0.2,facecolor='C0') for i in range(1,6,2)]
ax.plot(lpf.timea-tref, flux_pc[0], c='C0')
setp(ax, xlim=lpf.timea[[0,-1]]-tref, xlabel=f'Time - {tref:.0f} [BJD]', ylabel='Normalised flux')
fig.tight_layout()

az = fig.add_axes([0.075,0.18,0.20,0.46])
ax.add_patch(Rectangle((zx1,zy1),zx2-zx1,zy2-zy1,fill=False,edgecolor='k',lw=1,ls='dashed'))
[az.fill_between(lpf.timea-tref,*flux_pc[i:i+2,:],alpha=0.2,facecolor='C0') for i in range(1,6,2)]
setp(az, xlim=(zx1,zx2), ylim=(zy1,zy2), yticks=[], xticks=[])
az.plot(lpf.timea-tref, flux_pc[0], c='C0');

/var/folders/jj/jw0h86kd4q78t3rcm5127g8c0000gn/T/ipykernel_5162/2693491668.py:7: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  setp(ax, xlim=lpf.timea[[0,-1]]-tref, xlabel=f'Time - {tref:.0f} [BJD]', ylabel='Normalised flux')


We could (should) also plot the residuals, but I've left them out from the plot for clarity. The plot looks fine, and we can continue to have a look at the parameter estimates.

## Analysis

We start the analysis by making a Pandas data frame ``df``, using the ``df.describe`` to gen an overview of the estimates, and plotting the posteriors for the most interesting parameters as violin plots.

In [45]:
pd.set_option('display.precision',4)
df = lpf.posterior_samples(derived_parameters=True)
df.describe()

,tc,p,rho,b,k2,q1_r,q2_r,wn_loge_0,k,a,inc,t14,t23
count,1.2500e+04,1.2500e+04,12500.0000,12500.0000,12500.0000,12500.0000,12500.0000,12500.0000,12500.0000,12500.0000,12500.0000,12500.0000,10500.0000
mean,2.4585e+06,1.0914e+00,13.0726,0.5122,0.0209,0.4341,0.4951,-2.1361,0.1394,8.7824,1.5011,0.0435,0.0305
std,1.8390e-02,1.0017e-09,7.7173,0.2896,0.0104,0.2995,0.2918,1.1771,0.0387,2.4613,0.0646,0.0238,0.0147
min,2.4585e+06,1.0914e+00,0.1673,0.0396,0.0047,0.0191,0.0014,-3.9978,0.0685,2.1922,1.2265,0.0145,0.0027
25%,2.4585e+06,1.0914e+00,5.7712,0.3044,0.0116,0.1534,0.2672,-3.1503,0.1076,7.1363,1.4839,0.0332,0.0222
50%,2.4585e+06,1.0914e+00,14.0256,0.4741,0.0199,0.4046,0.4558,-2.3341,0.1410,9.5934,1.5135,0.0366,0.0259
75%,2.4585e+06,1.0914e+00,20.3606,0.7214,0.0293,0.7032,0.7593,-0.9201,0.1711,10.8637,1.5379,0.0490,0.0356
max,2.4585e+06,1.0914e+00,24.5164,0.9993,0.0394,0.9873,0.9919,-0.0965,0.1984,11.5576,1.5667,0.1616,0.0910


Posterior density for transit center:

In [46]:
df.tc.hist(bins=50)

<Axes: >

Plot the density for other parameters

In [22]:
fig, axs = subplots(1,4, figsize=(13,3))
pars = 'tc rho b k'.split()
[sb.violinplot(y=df[p], inner='quartile', ax=axs.flat[i]) for i,p in enumerate(pars)]
[axs.flat[i].text(0.05,0.9, p, transform=axs.flat[i].transAxes) for i,p in enumerate(pars)]
setp(axs, xticks=[], ylabel='')
fig.tight_layout()

While we're at it, let's plot some correlation plots. The limb darkening coefficients are correlated, and we'd also expect to see a correlation between the impact parameter and radius ratio.

All of our parameters are correlated

In [ ]:
ccols = ['k', 'rho', 'b', 'q1_r', 'q2_r']
corner(df[ccols].values, labels=ccols);

### <span style="color:darkblue">Questions and exercises</span>

1. Estimate the radius of the planet using the planet-star radius ratio (k) and the stellar radius (that you need to find from somewhere) Give the answer in Jupiter radii.

From the planet-star radius ratio $k = R_{p}/R_{\star}$ it is possible to obtain the planet radius $R_{p}$ as

$$
R_{p} = R_{\star}*k,
$$

where the stellar radius is $R_{\star} = 1.657 R_{\odot}$ [Collins et al., 2017].

In [48]:
R_p = np.array(df.k)*1.657 * u.R_sun
print("R_p = ", R_p.mean().value, "+/-", R_p.std())

R_p =  0.23091637703038734 +/- 0.0641172478105305 solRad


2. Compare the stellar density ($\rho$) estimated from the transit modelling to the theoretical stellar density for a WASP-12-like star.

The theoretical stellar density estimated by [Collins et al., 2017] for the star WASP-12 is:

$$
\rho_\star = 0.446 \pm 0.015\ \mathrm{g}\ \mathrm{cm^{-3}}.
$$

From the transit modelling that has been done in this work, the resulting stellar density is:

In [ ]:
rho = np.array(df.rho) * u.g / u.cm**3
print("rho =", rho.mean().value, "+/-", rho.std())

rho = 13.072620321230561 +/- 7.717028508754181 g / cm3


3. From the plot above you can see that many of the parameters are correlated with each other. What does this mean in practice?

If the model parameters are correlated, this means they are not independent from each other, and consequently the changes in one of them are systematically associated with changes in the other one. This dependencies have to be taken into consideration when interpreting the results, as it may leed to uncertainties or degeneracies.


---
<center>&copy;2025 Hannu Parviainen</center>